## Importing libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re

## Creating the CSV file to be written

In [2]:
with open("oxnard_foodfacility_insplist.csv", "w") as f:
    f.write("FacilityName,URL,Address,Phone,LastInspectionDate\n" )

## Defining a function that will take URL of each list, and extract the required details

In [3]:
def openPages(url):
    
    # gets the URL for each list-page of results
    page = requests.get(url)
    # converts the HTML of that URL into text
    soup = BeautifulSoup(page.text)   
    # finds the main-content div tag in the list-page
    maindiv = soup.find("div", {"class": "main-content"})    
    # finds all the a-tags in that div
    atags = maindiv.findAll("a")  
    # loop begins to run through first 50 a-tags; the remaining 2 a-tags in the end are irrelevant 
    for atag in atags[:50]:        
        # takes the name from each a-tag
        name = atag.text.strip()       
        # replaces commas with spaces from the name: this is done so that the second part of the name does not get 
        # written into the next column in the CSV
        name = name.replace(",", " ")
        print(name)        
        # URL for each result
        resurl = "http://www.decadeonline.com/" + atag["href"]      
        # gets the URL for each individual result
        detailpage = requests.get(resurl)
        # converts the HTML of that URL into text
        detailsoup = BeautifulSoup(detailpage.text)         
        # stores the name of the restaurant again: this is done because the CSV file will be written inside the loop
        # and not outside
        resname = name        
        # finds the main-content div tag in the result-page
        detmaindiv = detailsoup.find('div', {"class": "main-content"})        
        # checks if there is at all a main-content div tag: if no inspections were done, there will not be one
        if detmaindiv != None:            
            # finds the number of tables in the div: the number will depend on how many departments a facility has
            tables = detmaindiv.findAll("table")            
            # checks if a facility has only one department
            if len(tables) == 3:               
                # finds the td tag which stores the date
                datetds = detmaindiv.findAll("td",{"width": "16%"})                
                # stores the last stored date
                lastinsp = datetds[0].text.strip()                
                # finds the td tag which stores the contact details
                detailtds = detmaindiv.findAll("td",{"width": "65%"})                
                # stores the address which comes from two rows
                address = detailtds[1].text.strip() + " " + detailtds[2].text.strip()
                # spacing between address is removed
                address = " ".join(re.split("\s+", str(address), flags = re.UNICODE))
                # replaces commas with spaces from the address: this is done so that the second part of 
                # the address does not get written into the next column in the CSV
                address = address.replace(",", " ")
                # stores the phone number
                phone = detailtds[3].text.strip()
                # stores the link to the inspection page
                link = resurl                
                print(address)
                print(phone)
                print(link)
                print(lastinsp)                
                # appends the CSV
                with open("oxnard_foodfacility_insplist.csv", "a") as f:
                    f.write(resname + "," + link + "," + address + "," + phone + "," + lastinsp + "\n")            
            # checks if a facility will have more than one department which will mean separate pages for each department         
            elif len(tables) == 2:                
                # finds all the a-tags that links to each department
                sectionlinks = tables[1].findAll('a')               
                # sets up loop for going through each of the a-tags
                for sectionlink in sectionlinks:                   
                    # gets URL for each section
                    sectionurl = "http://www.decadeonline.com/" + sectionlink["href"]                   
                    # stores the URL: the CSV will have the link to the whole facility
                    link = sectionurl                 
                    # takes the URL from the section-page
                    sectionpage = requests.get(sectionurl)
                    # converts the HTML of that URL into text
                    sectionsoup = BeautifulSoup(sectionpage.text)                    
                    # finds the main-content div-tags in that URL
                    mainsecdiv = sectionsoup.find("div", {"class": "main-content"})                    
                    # checks if there is at all a main-content div tag: if no inspections were done, there will not be one 
                    if mainsecdiv != None:                        
                        # stores details, exactly like the previous case
                        lastinsp = mainsecdiv.findAll("td",{"width": "16%"})[0].text.strip()
                        address = mainsecdiv.findAll("td",{"width": "65%"})[1].text.strip() + " " + mainsecdiv.findAll("td",{"width": "65%"})[2].text.strip()
                        address = " ".join(re.split("\s+", str(address), flags = re.UNICODE))
                        address = address.replace(",", " ")
                        phone = mainsecdiv.findAll("td",{"width": "65%"})[3].text.strip()
                        print(address)
                        print(phone)
                        print(link)
                        print(lastinsp)
                        # appends the CSV
                        with open("oxnard_foodfacility_insplist.csv", "a") as f:
                            f.write(resname + "," + link + "," + address + "," + phone + "," + lastinsp + "\n")                    
                    # if no inspections were done: the section-page will not have a main-content div tag
                    else:
                        # stores details
                        address = "N/A"
                        phone = "N/A"
                        lastinsp = "Inspection Not Done"
                        print(address)
                        print(phone)
                        print(link)
                        print(lastinsp)
                        # appends the CSV
                        with open("oxnard_foodfacility_insplist.csv", "a") as f:
                            f.write(resname + "," + link + "," + address + "," + phone + "," + lastinsp + "\n")        
            # if no inspections were done: the page will not have a main-content div tag
            else: 
                link = resurl
                address = "N/A"
                phone = "N/A"
                lastinsp = "Inspection Not Done"
                print(address)
                print(phone)
                print(link)
                print(lastinsp)
                # appends the CSV
                with open("oxnard_foodfacility_insplist.csv", "a") as f:
                    f.write(resname + "," + link + "," + address + "," + phone + "," + lastinsp + "\n")

## Setting up a loop that runs through each result list

In [ ]:
# the loop uses the offset in the URL which starts from 0 and increases by increments of 50
for offset in range(0, 1350, 50):
    url = "http://www.decadeonline.com/results.phtml?agency=VEN&offset=" + str(offset) + "&businessname=&businessstreet=&city=OXNARD&zip=&facilityid=&FTS=&soundslike=&sort=FACILITY_NAME"
    # each time the loop runs, the offset increases by 50 and calls the function to extract the page details
    openPages(url)
    time.sleep(1)

1 TO 3 DOLLAR EXPRESS
436 E COLONIA RD OXNARD  CA 93030
805-760-1003
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0034412
10/22/2018
24 HOUR FITNESS
1651 S VICTORIA AVE OXNARD  CA 93035
805-988-6000
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0025253
08/01/2017
24 HOUR FITNESS
400 TOWNCENTER DR OXNARD  CA 93036
805-288-8509
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0033799
05/22/2018
4 WAY MEAT MARKET #6
440 N ROSE AVE OXNARD  CA 93030
805-486-0444
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0062379
02/05/2019
440 N ROSE AVE OXNARD  CA 93030
805-486-0444
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0062380
02/05/2019
440 N ROSE AVE OXNARD  CA 93030
805-486-0444
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0062378
02/05/2019
440 N ROSE AVE OXNARD  CA 93030
805-486-0444
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0062381
02/05/2019
4 WAY MEAT MARKET #7 HUENEME
541 W HUENEME RD OX

2958 STURGIS RD OXNARD  CA 93030
805-278-9395
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0002145
07/24/2018
ALCARAZ CATERING
2958 STURGIS RD OXNARD  CA 93030
805-278-9395
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0003925
06/26/2018
ALCARAZ CATERING
2958 STURGIS RD OXNARD  CA 93030
805-278-9395
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0009588
08/06/2018
ALCARAZ CATERING
2958 STURGIS RD OXNARD  CA 93030
805-278-9395
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0009085
11/21/2018
ALCARAZ CATERING
2958 STURGIS RD OXNARD  CA 93030
805-278-9395
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0003196
04/02/2018
ALCARAZ CATERING
2958 STURGIS RD OXNARD  CA 93030
805-278-9395
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0002730
07/30/2018
ALCARAZ CATERING
2958 STURGIS RD OXNARD  CA 93030
805-278-9395
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0031316
06/19/2018
ALCARAZ CATERING
2958 STURGIS RD OXNARD  CA 

2460 AUTO CENTER DR OXNARD  CA 93036
805-444-4475
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0049953
03/08/2019
AZUL
1450 S OXNARD BLVD OXNARD  CA 93030
805-233-4727
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0064095
09/27/2018
1450 S OXNARD BLVD OXNARD  CA 93030
805-233-4727
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0064094
10/04/2018
BABY'S BADASS BURGERS
2557 CORTEZ ST OXNARD  CA 93036
562-852-9210
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0036869
03/20/2018
BABY'S BADASS BURGERS
BABYS BADASS BURGERS
BAHIA RESTAURANT
349 S OXNARD BLVD OXNARD  CA 93030
805-487-5166
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0004654
04/02/2019
349 S OXNARD BLVD OXNARD  CA 93030
805-487-5166
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0000465
04/02/2019
BANANA ICE CREAM
1664 S OXNARD BLVD OXNARD  CA 93030
805-247-1891
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0021994
06/21/2018
B

1041 S OXNARD BLVD OXNARD  CA 93030
805-487-6933
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0003313
04/02/2019
CAFE AMRI
2000 OUTLET CENTER DR OXNARD  CA 93036
805-983-3026
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0024003
02/05/2019
CAFFEE
CAJUN COUNTRY CAFE
CAKE BAKE EAT
1500 GALLATIN PL OXNARD  CA 93030
805-366-1075
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0036070
08/16/2017
CALIFORNIA DEL SUR
CALIFORNIA DEL SUR
2360 EASTMAN AVE OXNARD  CA 93030
805-804-0496
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0021585
02/19/2019
CALIFORNIA DEL SUR (SC)
2360 EASTMAN AVE OXNARD  CA 93030
805-804-0496
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0002516
02/19/2019
CALIFORNIA'S PIZZA & DRINKS
CAMARILLO HIGH SCHOOL
CAMARILLO KETTLECORN AND SHAVED ICE
CAMARILLO KETTLECORN AND SHAVED ICE
CAMELOT ROOM LLC
1331 W GONZALES RD OXNARD  CA 93036
805-485-8502
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0040001
03/

1860 N VENTURA RD OXNARD  CA 93030
805-983-0999
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0001938
12/06/2018
CHEVRON - S & S
2901 SAVIERS RD OXNARD  CA 93033
805-483-9561
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0003052
10/25/2018
CHEVRON EXTRA MILE
1900 N ROSE AVE OXNARD  CA 93030
805-988-9099
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0004376
01/22/2019
CHEVRON OXNARD
877 S VENTURA RD OXNARD  CA 93030
805-984-8075
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0061470
10/10/2018
877 S VENTURA RD OXNARD  CA 93030
805-984-8075
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0060370
10/10/2018
CHICAGO HOTDOGS
2557 CORTEZ ST OXNARD  CA 93036
805-816-1716
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0037411
09/13/2018
CHICAGO LIQUOR
1930 N VENTURA RD OXNARD  CA 93036
805-485-1716
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0032560
01/11/2019
1930 N VENTURA RD OXNARD  CA 93036
805-4

...

331 ESPLANADE DR OXNARD  CA 93030
206-318-8705
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0004342
11/28/2018
STEP ONE FOOD MART
1049 S VENTURA RD OXNARD  CA 93030
805-382-6856
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0022058
03/25/2019
STICKY SWEET CUPCAKES
672 WHALEN WY OXNARD  CA 93036
805-302-1538
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0035019
10/23/2017
STUDENT & FOOD SERVICES CENTER
4000 S ROSE AVE OXNARD  CA 93033
805-986-5802
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0023290
11/07/2018
SUBWAY
2811 SAVIERS RD OXNARD  CA 93033
805-487-2900
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0023851
04/03/2019
SUBWAY
645 S VENTURA RD OXNARD  CA 93030
805-744-3015
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0030561
01/02/2019
SUBWAY
1811 VENTURA BLVD OXNARD  CA 93010
805-981-7827
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0002345
01/09/2019
SUBWAY
600 S VICTORIA AVE OXNARD  CA 93035
805-98

601 W HUENEME RD OXNARD  CA 93033
805-488-2723
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0002157
12/04/2018
TAQUERIA CHAVOCHO
921 RICHMOND AVE OXNARD  CA 93030
805-483-6700
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0023007
04/02/2018
TAQUERIA CHAVOCHO
900 W WOOLEY RD OXNARD  CA 93030

http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0000739
12/20/2018
TAQUERIA CUERNAVACA
1801 E VENTURA BLVD OXNARD  CA 93036
805-981-4111
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0032816
05/16/2018
TAQUERIA EL CHAVOCHO
921 RICHMOND AVE OXNARD  CA 93030
805-830-3566
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0036331
09/21/2018
TAQUERIA EL CHAVOCHO
921 RICHMOND AVE OXNARD  CA 93030
805-483-6700
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0025410
03/20/2019
TAQUERIA EL TAPATIO #12
550 COLLECTION BLVD OXNARD  CA 93036
805-814-1058
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0036642
10/25/2018
TAQUERIA EL TAPATIO #2
4

622 S A ST OXNARD  CA 93030
805-483-6633
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0004074
06/27/2018
TONY'S BILLAR
111 E 5TH ST OXNARD  CA 93033
805-575-7199
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0028583
02/13/2018
TOOPAK OIL CORPORATION
105 N OXNARD BLVD OXNARD  CA 93030
805-483-2810
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0035276
02/21/2018
TOPA TOPA CREATIONS BY CECILE FAULCONER
1901 LENNOX CT OXNARD  CA 93030
805-985-5038
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0035572
12/31/2018
TOPPERS PIZZA PLACE
2100 SAVIERS RD OXNARD  CA 93030
805-388-2432
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0000281
08/27/2018
TOPPERS PIZZA PLACE
2701 PENINSULA RD OXNARD  CA 93035
805-984-9395
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0024799
01/07/2019
TOPPERS PIZZA PLACE
111 E GONZALES RD OXNARD  CA 93030
805-385-4437
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0004062
05/17/2018
TORTAS LA MA

N/A
N/A
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0057056
Inspection Not Done
N/A
N/A
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0057057
Inspection Not Done
VENTURA COUNTY UNITED SOCCER LEAGUE
VENTURA COUNTY YOUTH TRACK CLUB
VIDA Y ALEGRIA
2502 S SAVIERS RD OXNARD  CA 93033
805-827-2402
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0033460
12/17/2018
VIVE NATURAL
1836 SAVIERS RD OXNARD  CA 93036
805-486-4636
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0023632
02/02/2018
VIVE TU MEJOR SALUD
VIZZI TRUCK  LLC
VONS #1913
450 S VENTURA RD OXNARD  CA 93030
805-240-7914
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0004612
11/01/2018
450 S VENTURA RD OXNARD  CA 93030
805-240-7914
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0005010
11/01/2018
450 S VENTURA RD OXNARD  CA 93030
805-240-7914
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0062467
11/01/2018
450 S VENTURA RD OXNARD  CA 93

2691 VENTURA BLVD OXNARD  CA 93036
805-485-4618
http://www.decadeonline.com/insp.phtml?agency=VEN&record_id=PR0005181
01/08/2019
XIELO ARTISAN DESSERTS
XIELO ARTISAN DESSERTS
212 W FOURTH ST OXNARD  CA 93030
805-890-1644
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0035332
02/12/2019
XIELO ARTISAN DESSERTS
YANIRA'S ICE CREAM
921 RICHMOND AVE OXNARD  CA 93030
805-844-8361
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0027488
08/29/2018
YANKEE DOODLE BAKERY
2035 N OXNARD BLVD OXNARD  CA 93036
805-485-7179
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0033524
11/22/2016
YANKEE DOODLE BAKERY
2035 N OXNARD BLVD OXNARD  CA 93030
805-485-7179
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0023745
10/10/2018
YANKEE DOODLE BAKERY (TRAILER)
2035 N OXNARD BLVD OXNARD  CA 93036
805-844-5260
http://www.decadeonline.com/fac.phtml?agency=VEN&facid=FA0023736
10/09/2018
YARD HOUSE
501 COLLECTION BLVD OXNARD  CA 93036
805-981-8707
http://www.decadeonline.com/i